<a href="https://colab.research.google.com/github/Mhanyn/n-queens-solver/blob/main/Nqueensproblem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Exhaustive Search => DFS**

In [ ]:
import time

def solve_n_queens(N):
    # Bitmask to represent columns and diagonals
    cols = 0
    diag1 = 0
    diag2 = 0
    solution_count = 0
    solutions = []

    # Array to track the position of queens
    queen_positions = [-1] * N
    configurations_explored = 0  # Counter for configurations explored

    def is_safe(row, col):
        bit = 1 << col
        return (cols & bit) == 0 and (diag1 & (bit << row)) == 0 and (diag2 & (bit << (N - 1 - row))) == 0

    def place_queen(row, col):
        bit = 1 << col
        nonlocal cols, diag1, diag2
        cols |= bit
        diag1 |= bit << row
        diag2 |= bit << (N - 1 - row)
        queen_positions[row] = col  # Store the column of the placed queen

    def remove_queen(row, col):
        bit = 1 << col
        nonlocal cols, diag1, diag2
        cols &= ~bit
        diag1 &= ~(bit << row)
        diag2 &= ~(bit << (N - 1 - row))
        queen_positions[row] = -1  # Reset the position

    def solve_util(row):
        nonlocal configurations_explored  # Access the outer variable
        if row == N:  # All queens are placed
            nonlocal solution_count
            solution_count += 1
            board = ['.' * N for _ in range(N)]
            for r in range(N):
                board[r] = board[r][:queen_positions[r]] + 'Q' + board[r][queen_positions[r]+1:]
            solutions.append(board)

        for col in range(N):
            configurations_explored += 1  # Count the configuration explored
            if is_safe(row, col):
                place_queen(row, col)
                solve_util(row + 1)  # Recursively place queens in the next rows
                remove_queen(row, col)

    solve_util(0)
    return solution_count, configurations_explored, solutions  # Return the additional data

def print_board(board):
    for row in board:
        print(" ".join(row))
    print()

# Generate and display the N-Queens solutions for N = 4
N = 10
start_time = time.time()
solution_count, configurations_explored, solutions = solve_n_queens(N)
end_time = time.time()

print(f"For N={N}, the number of possible solutions is {solution_count}")
print(f"Configurations explored: {configurations_explored}")
print(f"Time taken: {end_time - start_time:.6f} seconds")

if solutions:
    print("Displaying one of the solutions:")
    print_board(solutions[0])
else:
    print(f"No solution exists for N={N}.")

For N=10, the number of possible solutions is 724
Configurations explored: 355390
Time taken: 0.226100 seconds
Displaying one of the solutions:
Q . . . . . . . . .
. . Q . . . . . . .
. . . . . Q . . . .
. . . . . . . Q . .
. . . . . . . . . Q
. . . . Q . . . . .
. . . . . . . . Q .
. Q . . . . . . . .
. . . Q . . . . . .
. . . . . . Q . . .



**Genetic Algorithm**

In [ ]:
import random
import time

def generate_solution(N):
    """Generates a random solution for N-Queens."""
    return random.sample(range(N), N)  # Random permutation of columns

def fitness(solution):
    """Calculates the fitness of a solution."""
    N = len(solution)
    threats = 0
    row_conflicts = [0] * N
    col_conflicts = [0] * N
    diag1_conflicts = [0] * (2 * N - 1)  # row - col + (N - 1)
    diag2_conflicts = [0] * (2 * N - 1)  # row + col

    for row in range(N):
        col = solution[row]
        row_conflicts[row] += 1
        col_conflicts[col] += 1
        diag1_conflicts[row - col + (N - 1)] += 1
        diag2_conflicts[row + col] += 1

    for i in range(N):
        if row_conflicts[i] > 1:
            threats += row_conflicts[i] - 1
        if col_conflicts[i] > 1:
            threats += col_conflicts[i] - 1
    for i in range(2 * N - 1):
        if diag1_conflicts[i] > 1:
            threats += diag1_conflicts[i] - 1
        if diag2_conflicts[i] > 1:
            threats += diag2_conflicts[i] - 1

    return (N * (N - 1) // 2) - threats  # More pairs with fewer threats means better fitness

def select(parents):
    """Selects parents for crossover based on fitness."""
    total_fitness = sum(fitness(ind) for ind in parents)
    selection_probs = [fitness(ind) / total_fitness for ind in parents]
    return random.choices(parents, weights=selection_probs, k=2)

def crossover(parent1, parent2):
    """Crossover two parents to produce offspring."""
    point = random.randint(1, len(parent1) - 1)
    child = parent1[:point] + [gene for gene in parent2 if gene not in parent1[:point]]
    return child

def mutate(solution):
    """Mutates a solution by swapping two queens' positions."""
    if random.random() < 0.1:  # 10% chance of mutation
        idx1, idx2 = random.sample(range(len(solution)), 2)
        solution[idx1], solution[idx2] = solution[idx2], solution[idx1]  # Swap positions
    return solution

def genetic_algorithm(N, generations=1000):
    """Runs the genetic algorithm to solve the N-Queens problem."""
    population_size = 50 + N * 10  # Dynamic population size based on N
    population = [generate_solution(N) for _ in range(population_size)]
    best_solution = None
    best_fitness = -1
    configurations_explored = 0  # Counter for explored configurations

    for _ in range(generations):
        new_population = []

        # Ensuring best solutions survive (elitism)
        population.sort(key=fitness, reverse=True)
        new_population.extend(population[:10])  # Keep the best 10 solutions

        while len(new_population) < population_size:
            parent1, parent2 = select(population)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.extend([child1, child2])

        # Update the configurations explored based on the new population
        configurations_explored += len(population)  # All current population configurations are explored
        population = new_population

        # Updating best solution
        current_best = max(population, key=fitness)
        current_fitness = fitness(current_best)
        if current_fitness > best_fitness:
            best_solution = current_best
            best_fitness = current_fitness

    return best_solution, best_fitness, configurations_explored

if __name__ == "__main__":
    N = 14
    start_time = time.time()
    solution, best_fitness, configurations = genetic_algorithm(N)
    end_time = time.time()

    print(f"For N={N}, the best solution found is: {solution}")
    print(f"Best fitness: {best_fitness} (1 means valid placement of N queens)")
    print(f"Configurations explored: {configurations}")
    print(f"Time taken: {end_time - start_time:.6f}")

For N=14, the best solution found is: [9, 3, 12, 4, 2, 0, 13, 11, 8, 6, 1, 10, 5, 7]
Best fitness: 91 (1 means valid placement of N queens)
Configurations explored: 190000
Time taken: 432.758618
